# Aprendizaje Automatizado enfocado en la predicción de fallas de equipos DESAFIO ECOPETROL

### Desafío 000527: ¿Cómo aportar a la transformación energética, dinámicas sociales, culturales o tecnológicas?

## Machine Learning for Equipment Failure Prediction and Equipment Maintenance (PM)

Al trabajar con máquinas o instrumentos que requieren mantenimiento periódico, hay generalmente 3 posibles sucesos:

1. Puedes realizar realizar un mantenimiento demasiado frecuente. Es decir, se realiza mantenimiento cuando no es requerido. En este escenario estas perdiendo dinero. (Mantenimiento Preventivo No Eficiente)

1. No realizas mantenimiento con suficiente frecuencia. La máquina o instrumento terminara dañandose causando afectaciones a personas, ambiente y generando costos adicionales en reparación, producción, etc. (Mantenimiento Correctivo)

1. Se realiza el mantenimiento cuando se requiere. Mejor opción pero díficil de predecir el momento justo para realizar el mantenimiento. (Mantenimiento Preventivo Eficiente)

Los datos utilizados son artificiales pero basados en la experiencia de 4 años de estudio de fallas de equipos realizado por [Shad Griffin](https://medium.com/swlh/machine-learning-for-equipment-failure-prediction-and-predictive-maintenance-pm-e72b1ce42da1#)

La compañía problema o del caso de estudio posee datos que incluyen 419 máquinas que fallaron en el transcurso de 2 años. Generando 11.7 millones de dolares en gasto de mantenimiento. La mayoría de estos costos fueron de mantenimiento tipo correctivo.  

Actualmente le cuesta el mantenimiento a la compañía $28,000 USD por falla o por mantenimiento por máquina. 

**Meta: Bajar costo**

# <a id='0'>Tabla de Contenidos</a>

1. [Carga de Datos](#setup1)<br>
 
2. [Exploración de Datos](#explore)<br>

3. [Transformación de Datos e Ingeniería de Características](#trans)<br>
 
4. [Manejando el pequeño número de fallas](#small)<br>
    4.1 [Expandiendo la ventana de fallas](#window)<br>
    4.2 [Creación de pruebas, entrenamiento y grupos de validación](#groups)<br>
    4.3 [SMOTE the Training Data](#smote)<br>
5. [More Data Transformations and Feature Engineering](#more)<br>
6. [Build the Model on the Balanced Data Set](#build)<br>
7. [Score the Unbalanced Training Data Set](#score)<br>
8. [Business Rules and Heuristics](#bus)<br>
9. [Define a True Positive, True Negative, False Positive and False Negative](#tp)<br>
10. [Apply Model and Heuristics to the Testing and Validation Data Sets](#apply)<br>
11. [Conclusions](#conc)<br>

## 1.0 Carga de Datos <a id="setup1"></a>

 Se instalan e importan las librerias necesarias para el proyecto

In [9]:
!pip install imblearn --upgrade
!pip install plotly --upgrade
!pip install chart-studio --upgrade

Requirement already up-to-date: plotly in c:\users\user\anaconda3\lib\site-packages (4.14.3)


In [11]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
#from botocore.client import Config
#import ibm_boto3
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import types
import pandas as pd
#from botocore.client import Config
#import ibm_boto3

def __iter__(self): return 0

Importar la base de datos

In [14]:
# Convertir csv a un  df de pandas
df1 = pd.read_csv(r"C:\Users\user\Desktop\Profundización DS\Desafio Ecopetrol\equipment_failure_data_1.csv", sep=",", header=0)

In [15]:
# Convertir csv a un  df de pandas
df2 = pd.read_csv(r"C:\Users\user\Desktop\Profundización DS\Desafio Ecopetrol\equipment_failure_data_2.csv", sep=",", header=0)

In [16]:
#concatenación de los dos df en uno
df=pd.concat([df1,df2])

<a href="#0"><font size="1">Volver al inicio</font></a>
## 2.0 Exploración de Datos <a id="explore"></a>

In [17]:
df.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100001,12/2/14,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880
1,100001,12/3/14,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881
2,100001,12/4/14,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882
3,100001,12/5/14,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883
4,100001,12/6/14,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884


A continuación se explica la metadata de las columnas:

ID — Este campo representa una máquina en especifico.

DATE — Fecha de la observación.

REGION_CLUSTER — Campo que representa la región en la cuál la máquina se encuentra.

MAINTENANCE_VENDOR — Representa la compañía que provee el mantenimiento y servicio de la máquina.

MANUFACTURER — Compañía que manufacturó el equipo.

WELL_GROUP — Representa el tipo de máquina.

EQUIPMENT_AGE — Edad de la máquina en días.

S15 — Valor del sensor.

S17 — Valor del sensor.

S13 — Valor del sensor.

S16 — Valor del sensor.

S19 — Valor del sensor.

S18 — Valor del sensor.

S8 — Valor del sensor.

EQUIPMENT_FAILURE — Un ‘1’ significa que el equipo ha fallado y un ‘0’ que no lo ha hecho.

Nuestra meta es construir un modelo que prediga la falla del equipo, es decir, se utilizaran las otras variables para predecir EQUIPMENT_FAILURE.

Examinamos la cantidad de datos, filas y columnas. Son 307.751 filas y 16 columnas.

In [18]:
df.shape

(307751, 16)

Hay 421 máquinas en la base de datos (df)

In [21]:
df.ID.nunique()

421

Hay 731 datos unicos en la base de datos

In [27]:
idC = pd.DataFrame(df.groupby(['ID']).agg(['count']))
idC.sample(5)

,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
,count,count,count,count,count,count,count,count,count,count,count,count,count,count,count
ID,,,,,,,,,,,,,,,
100066,731,731,731,731,731,731,731,731,731,731,731,731,731,731,731
100310,731,731,731,731,731,731,731,731,731,731,731,731,731,731,731
100503,731,731,731,731,731,731,731,731,731,731,731,731,731,731,731
100530,731,731,731,731,731,731,731,731,731,731,731,731,731,731,731
100519,731,731,731,731,731,731,731,731,731,731,731,731,731,731,731


In [28]:
dateC = pd.DataFrame(df.groupby(['DATE']).agg(['count']))
dateC.sample(5)

,ID,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
,count,count,count,count,count,count,count,count,count,count,count,count,count,count,count
DATE,,,,,,,,,,,,,,,
8/5/16,421,421,421,421,421,421,421,421,421,421,421,421,421,421,421
4/13/16,421,421,421,421,421,421,421,421,421,421,421,421,421,421,421
1/19/16,421,421,421,421,421,421,421,421,421,421,421,421,421,421,421
2/11/15,421,421,421,421,421,421,421,421,421,421,421,421,421,421,421
10/12/15,421,421,421,421,421,421,421,421,421,421,421,421,421,421,421


Tenemos 731 datos únicos.  Entonces si tenemos 421 máquinas y 731 datos únicos, deberíamos tener 307.751 registros. Basados en el comando **.shape**, tenemos 1 registro por máquina por valor de registro. No hay duplicados en el **df**. De todas formas se utilizaran las funciones para comprobar esto.

In [30]:
duplicates = df.duplicated()
duplicates.value_counts()

False    307751
dtype: int64

Ahora buscaremos valores no registrados en los campos.

In [31]:
df.isnull().sum(axis = 0)

ID                    0
DATE                  0
REGION_CLUSTER        0
MAINTENANCE_VENDOR    0
MANUFACTURER          0
WELL_GROUP            0
S15                   0
S17                   0
S13                   0
S5                    0
S16                   0
S19                   0
S18                   0
EQUIPMENT_FAILURE     0
S8                    0
AGE_OF_EQUIPMENT      0
dtype: int64

Ahora examinamos la variable dependiente en más detalle. De los 307.751 registros, solo ocurrieron 421 fallas. Es decir, hay una tasa de falla del .14%. En otras palabras, por cada falla hay un poco más de 700 no-fallas. 
**Esto quiere decir que la base de datos está muy desbalanceada.** 

In [35]:
PFalla = df.EQUIPMENT_FAILURE.value_counts()[1]*100/df.EQUIPMENT_FAILURE.value_counts()[0]
PFalla

0.136986301369863

Se pueden explorar las variables numéricas descriptivas.

In [36]:
df.describe()

,ID,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
count,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000
mean,100310.826603,4.543943,14.585192,80.265541,35.018249,4675.848252,7.972097,9.069123,137.963064,0.001368,144.665715,2524.192399
std,177.574390,2.284121,8.817056,85.804273,14.446585,2521.074632,2.321949,16.898887,238.890128,0.036961,240.773926,3158.930976
min,100001.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.490000,0.000000
25%,100161.000000,3.000000,7.694100,0.000000,28.200000,3209.000000,6.621500,0.900000,11.798276,0.000000,9.250000,721.000000
50%,100311.000000,5.000000,11.661600,31.680000,34.940000,4237.047619,8.004000,4.200000,38.200000,0.000000,53.080000,1113.000000
75%,100467.000000,6.000000,22.560000,160.080000,41.610000,5743.000000,9.460000,10.600000,150.900000,0.000000,165.092608,2784.000000
max,100617.000000,8.000000,59.040000,2555.520000,592.890000,52767.000000,24.600000,511.000000,4151.700000,1.000000,2068.110000,15170.000000


Ahora se examina una correlación simple de la variable independiente con la dependiente.

In [40]:
corr=df.corr(method='pearson')

corr=corr[['EQUIPMENT_FAILURE']]
corr['ABS_EQUIPMENT_FAILURE']=abs(corr['EQUIPMENT_FAILURE'])
corr=corr.sort_values(by=['ABS_EQUIPMENT_FAILURE'], ascending=[False])
corr

,EQUIPMENT_FAILURE,ABS_EQUIPMENT_FAILURE
EQUIPMENT_FAILURE,1.000000e+00,1.000000e+00
S15,-6.036352e-02,6.036352e-02
S17,-3.429070e-02,3.429070e-02
S18,9.765002e-03,9.765002e-03
S13,-8.617761e-03,8.617761e-03
S5,-7.189979e-03,7.189979e-03
S8,6.517148e-03,6.517148e-03
S16,-6.138895e-03,6.138895e-03
S19,-6.087474e-03,6.087474e-03
AGE_OF_EQUIPMENT,4.733368e-04,4.733368e-04


<a href="#0"><font size="1">Volver al inicio</font></a>
## 3.0 Transformación de Datos e Ingeniería de Características <a id="trans"></a>

Next, we can transform our data for a machine learning model. Specifically, we will create running summaries of the sensor values. Running summaries of sensor values are often useful in predicting equipment failure. For example, if a temperature gauge indicates a machine is warmer than average for the last five days, it may mean something is wrong.

Remember that we are working with a panel data set. That is, we have multiple machines measured over two years. As we create our running summaries, we have to make sure that our summaries do not include more than one machine. For example, if we create a ten-day moving average, we do not want the first nine days of a machine to have values from the previous machine.

Note that I create twenty-one-day summaries in this example. This works for this use case, but it may be advantageous to use more or different time intervals for other situations.


Convert dates from character to date.

In [ ]:


pd_data['DATE'] = pd.to_datetime(pd_data['DATE'])





Create a new field called “flipper” that indicates when the id changes as the data are sorted by ID and DATE in ascending order. We will use this in a few other transformations.

In [ ]:
pd_data=pd_data.sort_values(by=['ID','DATE'], ascending=[True, True])

pd_data['flipper'] = np.where((pd_data.ID != pd_data.ID.shift(1)), 1, 0)
pd_data.head()

Running summaries are often useful transformations for these types of a problems.  For example, a running mean would be the average value over the last x days.  X in this case is the feature window.  The feature window is a parameter that depends on the context of the business problem.  I am setting the value to 21 days, but this may or may not work for your business problem.

In [ ]:
#define your feature window. This is the window by which we will aggregate our sensor values.
feature_window=21

Calculate the number of days from the first day a machine appears to the current day. This field will be called “TIME_SINCE_START” Also, create a variable called “too_soon.” When “too_soon” is equal to 1, we have less than 21 days (feature_window) of history for the machine.

We will use these new variables to create a running mean, median, max, and min. 


In [ ]:
dfx=pd_data

In [ ]:
#Select the first record of each machine

starter=dfx[dfx['flipper'] == 1]

starter=starter[['DATE','ID']]

In [ ]:
#rename data to start_date
starter=starter.rename(index=str, columns={"DATE": "START_DATE"})

In [ ]:
#convert START_DATE to date
starter['START_DATE'] = pd.to_datetime(starter['START_DATE'])

In [ ]:
#Merge START_DATE to the original data set

dfx=dfx.sort_values(by=['ID', 'DATE'], ascending=[True, True])
starter=starter.sort_values(by=['ID'], ascending=[True])
dfx =dfx.merge(starter, on=['ID'], how='left')

In [ ]:
# calculate the number of days since the beginning of each well. 
dfx['C'] = dfx['DATE'] - dfx['START_DATE']
dfx['TIME_SINCE_START'] = dfx['C'] / np.timedelta64(1, 'D')
dfx=dfx.drop(columns=['C'])
dfx['too_soon'] = np.where((dfx.TIME_SINCE_START < feature_window) , 1, 0)


Create a running mean, max, min and median for the sensor variables.


In [ ]:
dfx['S5_mean'] = np.where((dfx.too_soon == 0),(dfx['S5'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S5)
dfx['S5_median'] = np.where((dfx.too_soon == 0),(dfx['S5'].rolling(min_periods=1, window=feature_window).median()) , dfx.S5)
dfx['S5_max'] = np.where((dfx.too_soon == 0),(dfx['S5'].rolling(min_periods=1, window=feature_window).max()) , dfx.S5)
dfx['S5_min'] = np.where((dfx.too_soon == 0),(dfx['S5'].rolling(min_periods=1, window=feature_window).min()) , dfx.S5)


dfx['S13_mean'] = np.where((dfx.too_soon == 0),(dfx['S13'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S13)
dfx['S13_median'] = np.where((dfx.too_soon == 0),(dfx['S13'].rolling(min_periods=1, window=feature_window).median()) , dfx.S13)
dfx['S13_max'] = np.where((dfx.too_soon == 0),(dfx['S13'].rolling(min_periods=1, window=feature_window).max()) , dfx.S13)
dfx['S13_min'] = np.where((dfx.too_soon == 0),(dfx['S13'].rolling(min_periods=1, window=feature_window).min()) , dfx.S13)


dfx['S15_mean'] = np.where((dfx.too_soon == 0),(dfx['S15'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S15)
dfx['S15_median'] = np.where((dfx.too_soon == 0),(dfx['S15'].rolling(min_periods=1, window=feature_window).median()) , dfx.S15)
dfx['S15_max'] = np.where((dfx.too_soon == 0),(dfx['S15'].rolling(min_periods=1, window=feature_window).max()) , dfx.S15)
dfx['S15_min'] = np.where((dfx.too_soon == 0),(dfx['S15'].rolling(min_periods=1, window=feature_window).min()) , dfx.S15)

dfx['S16_mean'] = np.where((dfx.too_soon == 0),(dfx['S16'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S16)
dfx['S16_median'] = np.where((dfx.too_soon == 0),(dfx['S16'].rolling(min_periods=1, window=feature_window).median()) , dfx.S16)
dfx['S16_max'] = np.where((dfx.too_soon == 0),(dfx['S16'].rolling(min_periods=1, window=feature_window).max()) , dfx.S16)
dfx['S16_min'] = np.where((dfx.too_soon == 0),(dfx['S16'].rolling(min_periods=1, window=feature_window).min()) , dfx.S16)


dfx['S17_mean'] = np.where((dfx.too_soon == 0),(dfx['S17'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S17)
dfx['S17_median'] = np.where((dfx.too_soon == 0),(dfx['S17'].rolling(min_periods=1, window=feature_window).median()) , dfx.S17)
dfx['S17_max'] = np.where((dfx.too_soon == 0),(dfx['S17'].rolling(min_periods=1, window=feature_window).max()) , dfx.S17)
dfx['S17_min'] = np.where((dfx.too_soon == 0),(dfx['S17'].rolling(min_periods=1, window=feature_window).min()) , dfx.S17)

dfx['S18_mean'] = np.where((dfx.too_soon == 0),(dfx['S18'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S18)
dfx['S18_median'] = np.where((dfx.too_soon == 0),(dfx['S18'].rolling(min_periods=1, window=feature_window).median()) , dfx.S18)
dfx['S18_max'] = np.where((dfx.too_soon == 0),(dfx['S18'].rolling(min_periods=1, window=feature_window).max()) , dfx.S18)
dfx['S18_min'] = np.where((dfx.too_soon == 0),(dfx['S18'].rolling(min_periods=1, window=feature_window).min()) , dfx.S18)



dfx['S19_mean'] = np.where((dfx.too_soon == 0),(dfx['S19'].rolling(min_periods=1, window=feature_window).mean()) , dfx.S19)
dfx['S19_median'] = np.where((dfx.too_soon == 0),(dfx['S19'].rolling(min_periods=1, window=feature_window).median()) , dfx.S19)
dfx['S19_max'] = np.where((dfx.too_soon == 0),(dfx['S19'].rolling(min_periods=1, window=feature_window).max()) , dfx.S19)
dfx['S19_min'] = np.where((dfx.too_soon == 0),(dfx['S19'].rolling(min_periods=1, window=feature_window).min()) , dfx.S19)


dfx.head()

Another useful transformation is to look for sudden spikes in sensor values. This code creates a value indicating how far the current value is from the immediate norm.

In [ ]:
dfx['S5_chg'] = np.where((dfx.S5_mean == 0),0 , dfx.S5/dfx.S5_mean)


dfx['S13_chg'] = np.where((dfx.S13_mean == 0),0 , dfx.S13/dfx.S13_mean)

dfx['S15_chg'] = np.where((dfx.S15_mean==0),0 , dfx.S15/dfx.S15_mean)
dfx['S16_chg'] = np.where((dfx.S16_mean == 0),0 , dfx.S16/dfx.S16_mean)
dfx['S17_chg'] = np.where((dfx.S17_mean == 0),0 , dfx.S17/dfx.S17_mean)
dfx['S18_chg'] = np.where((dfx.S18_mean == 0),0 , dfx.S18/dfx.S18_mean)
dfx['S19_chg'] = np.where((dfx.S19_mean == 0),0 , dfx.S19/dfx.S19_mean)

In [ ]:
#copy the data set to the original name
pd_data=dfx

## 4.0 Dealing with the small number of failures. <a id="small"></a>

### 4.1 Expand the Failure (Target) Window <a id="window"></a>

Machines are engineered to last. If something breaks all the time, you won’t buy it, would you?

Because machines generally last a long time, we typically do not have many examples of failure. This means the data sets we use in PM are almost always unbalanced. 

One way to increase the number of failures is to expand the failure or target window. That is, make the dependent variable, not just the day the equipment failed but the 30 days (or another appropriate interval) leading up to the failure.

In this example, I use a 28-day target window. We will use the 28 days leading up to a failure as the dependent variable in our model.


In [ ]:
target_window=28

Sort the data by ID and DATE.  Make sure the index reflects this order.

In [ ]:
pd_data=pd_data.sort_values(by=['ID', 'DATE'], ascending=[True, True])
pd_data.reset_index(level=0, inplace=True)

Create a new data frame that contains the failure records.  Rename DATE to FAILURE_DATE

In [ ]:


df_failure_thingy=pd_data[pd_data['EQUIPMENT_FAILURE'] == 1]

df_failure_thingy=df_failure_thingy[['DATE','ID']]

df_failure_thingy=df_failure_thingy.rename(index=str, columns={"DATE": "FAILURE_DATE"})

pd_data=pd_data.sort_values(by=['ID'], ascending=[True])
df_failure_thingy=df_failure_thingy.sort_values(by=['ID'], ascending=[True])


Append the FAILURE_DATE to each ID

In [ ]:


pd_data =pd_data.merge(df_failure_thingy, on=['ID'], how='left')

For each record calculate the number of days until failure.

In [ ]:


pd_data=pd_data.sort_values(by=['ID','DATE'], ascending=[True, True])

pd_data['FAILURE_DATE'] = pd.to_datetime(pd_data['FAILURE_DATE'])
pd_data['DATE'] = pd.to_datetime(pd_data['DATE'])
pd_data['C'] = pd_data['FAILURE_DATE'] - pd_data['DATE']

pd_data['TIME_TO_FAILURE'] = pd_data['C'] / np.timedelta64(1, 'D')

Clean up and sort the records by ID and DATE

In [ ]:
pd_data=pd_data.drop(columns=['index'])

In [ ]:
pd_data=pd_data.sort_values(by=['ID', 'DATE'], ascending=[True, True])



In [ ]:
pd_data.reset_index(inplace=True)


In [ ]:
pd_data.head()

Create a new variable, FAILURE_TARGET.  It is equal to 1 if the record proceeds a failure by "failure_window" days or less.

In [ ]:
pd_data['FAILURE_TARGET'] = np.where(((pd_data.TIME_TO_FAILURE < target_window) & ((pd_data.TIME_TO_FAILURE>=0))), 1, 0)

pd_data.head()

In [ ]:
tips_summed = pd_data.groupby(['FAILURE_TARGET'])['S5'].count()
tips_summed

The new field occurs about 4% of the time.

In [ ]:
pd_data['FAILURE_TARGET'].mean()

Now we have 11,740 target observations. This is better, but the data set is far from balanced. In the next section, we will use SMOTE to increase the number of failures synthetically. However, before we do that, let’s split our data into training, testing, and a validation sample.

### 4.2 Create the Testing, Training and Validation Groupings <a id="groups"></a>

Because we are dealing with a panel data set (cross-sectional time-series), it is better not to take a random sample of all records. Doing so would put the records from one machine in all three sample data sets. To avoid this, we’ll randomly select IDs and place all of the records for each machine in either the training, testing, or validation data set.

In [ ]:
#Get a Unique List of All IDs 


aa=pd_data

pd_id=aa.drop_duplicates(subset='ID')
pd_id=pd_id[['ID']]
pd_id.shape


Create a new variable with a random number between 0 and 1

In [ ]:
np.random.seed(42)

In [ ]:
pd_id['wookie'] = (np.random.randint(0, 10000, pd_id.shape[0]))/10000

In [ ]:

pd_id=pd_id[['ID', 'wookie']]

Give each record a 30% chance of being in the validation, a 35% chance of being in the testing and a 35% chance of being in the training data set


In [ ]:
pd_id['MODELING_GROUP'] = np.where(((pd_id.wookie <= 0.35)), 'TRAINING', np.where(((pd_id.wookie <= 0.65)), 'VALIDATION', 'TESTING'))

This is how many machines fall in each group

In [ ]:
tips_summed = pd_id.groupby(['MODELING_GROUP'])['wookie'].count()
tips_summed

Append the Group of each id to each individual record

In [ ]:
pd_data=pd_data.sort_values(by=['ID'], ascending=[True])
pd_id=pd_id.sort_values(by=['ID'], ascending=[True])

In [ ]:
pd_data =pd_data.merge(pd_id, on=['ID'], how='inner')

pd_data.head()

This is how many records are in each group.

In [ ]:
tips_summed = pd_data.groupby(['MODELING_GROUP'])['wookie'].count()
tips_summed

This is how many failure targets are in each group.

In [ ]:
tips_summed = pd_data.groupby(['MODELING_GROUP'])['FAILURE_TARGET'].sum()
tips_summed

Create a separate data frame for the training data.  We will use this data set to build the model.

In [ ]:
df_training=pd_data[pd_data['MODELING_GROUP'] == 'TRAINING']
df_training=df_training.drop(columns=['MODELING_GROUP','C','wookie','TIME_TO_FAILURE','flipper','START_DATE'])
df_training.shape

Create a separate data frame for the training and testing data sets.  We will use this to tweak our modeling results.

In [ ]:
df_train_test=pd_data[pd_data['MODELING_GROUP'] != 'VALIDATION']

df_train_test=df_train_test.drop(columns=['wookie','TIME_TO_FAILURE','flipper','START_DATE'])
df_train_test.shape

Create a separate data frame for all the data. We will use this to validate the model and comapre the accuracy of all groups.

In [ ]:
df_total=pd_data.drop(columns=['C','wookie','TIME_TO_FAILURE','flipper','START_DATE'])
df_total.shape

### 4.3 SMOTE the Training Data <a id="smote"></a>

Note that we are only balancing the training data set. You may be asking why. Remember that our goal is to build a model the represents reality, right? When we SMOTE the data, we change the failure rate to 50%. This is no where near what we see in the actual machine data. Thus, goodness of fit statistics can be skewed. Because of this, it makes sense to build the model on the SMOTE data, but validate and test it on on the unaltered data. The unaltered data will be a better reflection of what to expect when you deploy the model to production.

Define the Training features and Target

In [ ]:
training_features=df_training[['REGION_CLUSTER','MAINTENANCE_VENDOR','MANUFACTURER','WELL_GROUP','AGE_OF_EQUIPMENT','S15','S17','S13','S5',
 'S16','S19','S18','S8','S5_mean','S5_median','S5_max','S5_min','S13_mean','S13_median','S13_max','S13_min','S15_mean','S15_median',
 'S15_max','S15_min','S16_mean','S16_median','S16_max','S16_min','S17_mean','S17_median','S17_max','S17_min','S18_mean','S18_median','S18_max','S18_min','S19_mean','S19_median','S19_max','S19_min',
 'S5_chg','S13_chg','S15_chg','S16_chg','S17_chg','S18_chg','S19_chg']]

In [ ]:
training_target=df_training[['FAILURE_TARGET']]

Synthetically Balance the training data sets with a SMOTE algorithm. After we apply the SMOTE algorithm, we will have a balanced data set. 50% Failures and 50% Non-Failures. Note that this takes a while to run.

In [ ]:
#uncomment these options if you want to expand the number of rows and columns that appear visually on the screen.

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
#sm = SMOTE(random_state=12, ratio = 1.0)
smx = SMOTENC(random_state=12,  categorical_features=[0, 1, 2, 3])

In [ ]:
x_res, y_res = smx.fit_sample(training_features, training_target.values.ravel())

Convert the SMOTE output back to complete data frames with independent and dependent variables.  Examine the results.

Format the Independent Variables

In [ ]:
df_x=pd.DataFrame(x_res)

df_x.columns = [
 'REGION_CLUSTER','MAINTENANCE_VENDOR','MANUFACTURER','WELL_GROUP','AGE_OF_EQUIPMENT','S15','S17','S13','S5','S16','S19',
 'S18','S8','S5_mean','S5_median','S5_max','S5_min','S13_mean','S13_median','S13_max','S13_min','S15_mean','S15_median','S15_max',
 'S15_min','S16_mean','S16_median','S16_max','S16_min','S17_mean','S17_median','S17_max','S17_min','S18_mean','S18_median','S18_max','S18_min',
 'S19_mean','S19_median','S19_max','S19_min','S5_chg','S13_chg','S15_chg','S16_chg','S17_chg','S18_chg','S19_chg']
df_x.head()

Format the Dependent Variable

In [ ]:
df_y=pd.DataFrame(y_res)
df_y.columns = ['FAILURE_TARGET']

Check that the dependent variable is balanced.  It is.

In [ ]:
df_y.mean(axis = 0) 

Merge the dependent and independent variables post SMOTE into a dataframe

In [ ]:
df_balanced = pd.concat([df_y, df_x], axis=1)
df_balanced.head()

## 5.0 More data transformation and feature engineering <a id="more"></a>

Convert the categorical variables into binary dummy variables. We need to do this because the XGBT model (below) doesn't like categorical fields.

In [ ]:
df_dv = pd.get_dummies(df_balanced['REGION_CLUSTER'])

df_dv=df_dv.rename(columns={"A": "CLUSTER_A","B":"CLUSTER_B","C":"CLUSTER_C","D":"CLUSTER_D","E":"CLUSTER_E","F":"CLUSTER_F","G":"CLUSTER_G","H":"CLUSTER_H"})


df_balanced= pd.concat([df_balanced, df_dv], axis=1)


df_dv = pd.get_dummies(df_balanced['MAINTENANCE_VENDOR'])

df_dv=df_dv.rename(columns={"I": "MV_I","J":"MV_J","K":"MV_K","L":"MV_L","M":"MV_M","N":"MV_N","O":"MV_O","P":"MV_P"})


df_balanced = pd.concat([df_balanced, df_dv], axis=1)



df_dv = pd.get_dummies(df_balanced['MANUFACTURER'])

df_dv=df_dv.rename(columns={"Q": "MN_Q","R":"MN_R","S":"MN_S","T":"MN_T","U":"MN_U","V":"MN_V","W":"MN_W","X":"MN_X","Y":"MN_Y","Z":"MN_Z"})


df_balanced = pd.concat([df_balanced, df_dv], axis=1)


df_dv = pd.get_dummies(df_balanced['WELL_GROUP'])

df_dv=df_dv.rename(columns={1: "WG_1",2:"WG_2",3:"WG_3",4:"WG_4",5:"WG_5",6:"WG_6",7:"WG_7",8:"WG_8"})


df_balanced = pd.concat([df_balanced, df_dv], axis=1)


In [ ]:
df_dv = pd.get_dummies(df_train_test['REGION_CLUSTER'])

df_dv=df_dv.rename(columns={"A": "CLUSTER_A","B":"CLUSTER_B","C":"CLUSTER_C","D":"CLUSTER_D","E":"CLUSTER_E","F":"CLUSTER_F","G":"CLUSTER_G","H":"CLUSTER_H"})


df_train_test= pd.concat([df_train_test, df_dv], axis=1)


df_dv = pd.get_dummies(df_train_test['MAINTENANCE_VENDOR'])

df_dv=df_dv.rename(columns={"I": "MV_I","J":"MV_J","K":"MV_K","L":"MV_L","M":"MV_M","N":"MV_N","O":"MV_O","P":"MV_P"})


df_train_test = pd.concat([df_train_test, df_dv], axis=1)



df_dv = pd.get_dummies(df_train_test['MANUFACTURER'])

df_dv=df_dv.rename(columns={"Q": "MN_Q","R":"MN_R","S":"MN_S","T":"MN_T","U":"MN_U","V":"MN_V","W":"MN_W","X":"MN_X","Y":"MN_Y","Z":"MN_Z"})


df_train_test = pd.concat([df_train_test, df_dv], axis=1)


df_dv = pd.get_dummies(df_train_test['WELL_GROUP'])

df_dv=df_dv.rename(columns={1: "WG_1",2:"WG_2",3:"WG_3",4:"WG_4",5:"WG_5",6:"WG_6",7:"WG_7",8:"WG_8"})


df_train_test = pd.concat([df_train_test, df_dv], axis=1)



In [ ]:
df_dv = pd.get_dummies(df_total['REGION_CLUSTER'])

df_dv=df_dv.rename(columns={"A": "CLUSTER_A","B":"CLUSTER_B","C":"CLUSTER_C","D":"CLUSTER_D","E":"CLUSTER_E","F":"CLUSTER_F","G":"CLUSTER_G","H":"CLUSTER_H"})


df_total= pd.concat([df_total, df_dv], axis=1)


df_dv = pd.get_dummies(df_total['MAINTENANCE_VENDOR'])

df_dv=df_dv.rename(columns={"I": "MV_I","J":"MV_J","K":"MV_K","L":"MV_L","M":"MV_M","N":"MV_N","O":"MV_O","P":"MV_P"})


df_total = pd.concat([df_total, df_dv], axis=1)



df_dv = pd.get_dummies(df_total['MANUFACTURER'])

df_dv=df_dv.rename(columns={"Q": "MN_Q","R":"MN_R","S":"MN_S","T":"MN_T","U":"MN_U","V":"MN_V","W":"MN_W","X":"MN_X","Y":"MN_Y","Z":"MN_Z"})


df_total = pd.concat([df_total, df_dv], axis=1)


df_dv = pd.get_dummies(df_total['WELL_GROUP'])

df_dv=df_dv.rename(columns={1: "WG_1",2:"WG_2",3:"WG_3",4:"WG_4",5:"WG_5",6:"WG_6",7:"WG_7",8:"WG_8"})


df_total = pd.concat([df_total, df_dv], axis=1)

## 6.0 Build the model on the balanced training data set <a id="build"></a>

In [ ]:
# Remove the newly redundant categorical variables.  This are now represented by dummy variables.
df_balanced=df_balanced.drop(columns=['REGION_CLUSTER','MAINTENANCE_VENDOR','MANUFACTURER','WELL_GROUP'])

In the balanced data set, separate the dependent and independent variables to feed to the model development process.

In [ ]:

features = [x for x in df_balanced.columns if x not in ['FAILURE_TARGET','EQUIPMENT_FAILURE']]  
dependent=pd.DataFrame(df_balanced['FAILURE_TARGET'])

independent=df_balanced.drop(columns=['FAILURE_TARGET'])

In [ ]:
#make sure everything is numeric for simplicity
independent = independent.apply(pd.to_numeric) 
df_balanced = df_balanced.apply(pd.to_numeric)

Define model specs

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

def evaluate_model(alg, train, target, predictors,  early_stopping_rounds=1):
    
   
    #Fit the algorithm on the data
    alg.fit(train[predictors], target['FAILURE_TARGET'], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(train[predictors])
    dtrain_predprob = alg.predict_proba(train[predictors])[:,1]
    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False) 
    feat_imp.plot(kind='bar', title='Feature Importance', color='g') 
    plt.ylabel('Feature Importance Score')
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(target['FAILURE_TARGET'].values, dtrain_predictions))
    print("AUC Score (Balanced): %f" % metrics.roc_auc_score(target['FAILURE_TARGET'], dtrain_predprob))

We are initializing our model with default model parameters. Note that we could probably improve the results by tweaking the parameters, but we will save that exercise for another day. This step can take a while.

In [ ]:
xgb0 = XGBClassifier(
 objective= 'binary:logistic')

evaluate_model(xgb0, independent, dependent,features) 

Using a 50% cut-off on the balanced data, the accuracy is 84% and the AUC score is 91.6%

Now, we will apply the model to the unbalanced training and testing data set.

## 7.0 Evaluate model on the unbalanced training and testing data set. <a id="score"></a>

This will assign a probability to fail for each record in the unbalanced training data set.

In [ ]:
df_train_test['P_FAIL']= xgb0.predict_proba(df_train_test[features])[:,1];


Use a cut-off of .67 to categorize the p-value into a binary variable. If the probability of failing is greater than 67%, the record is a predicted failure. If the chance to fail is less than 67%, the record is a predicted non-failure. Note that determining the best cut-off is a bit art and a bit science. I will reserve a discussion on this for another day, but here are some techniques if you are interested.  
https://medium.com/swlh/determining-a-cut-off-or-threshold-when-working-with-a-binary-dependent-target-variable-7c2342cf2a7c


Also, split the training and testing records into two data frames.

In [ ]:
df_train_test['Y_FAIL'] = np.where(((df_train_test.P_FAIL <= .67)), 0, 1)


In [ ]:
df_testing=df_train_test[df_train_test['MODELING_GROUP'] == 'TESTING']

In [ ]:
df_training=df_train_test[df_train_test['MODELING_GROUP'] != 'TESTING']

Build a ROC Curve for Unbalanced Testing Data

In [ ]:
fpr, tpr, threshold=metrics.roc_curve(df_testing['FAILURE_TARGET'], df_testing['P_FAIL'])
roc_auc = metrics.auc(fpr, tpr)
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Build a ROC Curve for Unbalanced Training Data

In [ ]:
fpr, tpr, threshold=metrics.roc_curve(df_training['FAILURE_TARGET'], df_training['P_FAIL'])
roc_auc = metrics.auc(fpr, tpr)
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

With a cut-off of .67 the unblananced testing data set has an accuracy of 94% and AUC Score of 59%

In [ ]:
#Print model report:
print("Accuracy : %.4g" % metrics.accuracy_score(df_testing['FAILURE_TARGET'].values, df_testing['Y_FAIL']))
print("AUC Score (Test): %f" % metrics.roc_auc_score(df_testing['FAILURE_TARGET'], df_testing['P_FAIL']))

With a cut-off of .67 the unblananced training data set has an accuracy of 95% and AUC Score of 89%

In [ ]:
#Print model report:
print("Accuracy : %.4g" % metrics.accuracy_score(df_training['FAILURE_TARGET'].values, df_training['Y_FAIL']))
print("AUC Score (Train): %f" % metrics.roc_auc_score(df_training['FAILURE_TARGET'], df_training['P_FAIL']))

Based on the AUC curve, this model kind of stinks, huh? Let's take a look at the testing data with a confusion matrix.

First, we will examine ('FAILURE_TARGET') the variable that is '1' if a date is within 28 days of a failure.

Create a confusion Matrix on the Testing Data

In [ ]:
print(pd.crosstab(df_testing.Y_FAIL, df_testing.FAILURE_TARGET, dropna=False))
pd.crosstab(df_testing.Y_FAIL, df_testing.FAILURE_TARGET).apply(lambda r: r/r.sum(), axis=1)

This confusion matrix doesn’t work, does it? Think about it. 

For example, let’s say that the machine fails on Sunday, and we have failure signals on Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, and Sunday. According to the matrix above, we would register six false positives and one true positive. In reality, we have one signal that occurred on seven consecutive days. These seven days should be counted as one true positive.

Next, we will examine a confusion matrix with the original variable. A ‘1’ appears only on the days where a failure is registered

In [ ]:
print(pd.crosstab(df_testing.Y_FAIL, df_testing.EQUIPMENT_FAILURE, dropna=False))
pd.crosstab(df_testing.Y_FAIL, df_testing.EQUIPMENT_FAILURE).apply(lambda r: r/r.sum(), axis=1)

This confusion matrix doesn’t work either, does it?

Think about it. A real positive only occurs if a failure happens on the same day as the signal. What if a signal happens on Monday, and it failed on Tuesday? I would count that as a true positive, but the confusion matrix above does not.

So, we have some cleaning up to do. In the next section, we will build a more realistic confusion matrix.



## 8.0 Business Rules and Heuristics <a id="bus"></a>

There is subjectivity when we create business rules and heuristics, just like when you fine-tune a model. For this reason, make sure you do your tweaking on the training data set and validate the results on the testing and validation data sets. This gives you the best chance of creating a result that will be reflective of an implementation.
First, we will create a business rule that eliminates signals in “bunches.” For example, if there is a signal on Monday, Tuesday, and Wednesday, you likely don’t have three different problems. You more than probably have one problem identified on three consecutive days.

To reduce the repetitive signals, we will create a business rule. This business rule will prohibit more than one signal every 90 days.

Essentially we are saying that a failure signal means the equipment will fail in the next 90 days. Determining the window depends on the frequency of failure. If the equipment fails ten times a year, a 90-day window isn’t very useful. On the other hand, if the equipment fails once every five years, a three-month window is meaningful. Remember, the machines for this exercise are designed to last 4 to 6 years.



In [ ]:
forecast_window=90

In [ ]:
#sort the data by id and date.
xx=df_training
xx=xx.sort_values(by=['ID','DATE'], ascending=[True, True])

Because our data is a panel, we must define the forecast window by machine ID. We want to prevent signals from one machine from "bleeding" into another machine. 

In other words, if a failure signal occurs on the last day a machine appears in the data, we don't want that signal to effect the subsequent machine.

The following code ensures that each forecast window is specific to each machine.

In [ ]:
#create a unique list of machines
aa=xx

pd_id=aa.drop_duplicates(subset='ID')
pd_id=pd_id[['ID']]
pd_id.shape

In [ ]:
#label each machine with a sequential number
pd_id=pd_id.reset_index(drop=True)
pd_id=pd_id.reset_index(drop=False)
pd_id=pd_id.rename(columns={"index": "SCOOBYDOO"})
pd_id['SCOOBYDOO']=pd_id['SCOOBYDOO']+1
pd_id.head()

In [ ]:
#grab the max number of machines +1

In [ ]:
column = pd_id["SCOOBYDOO"]
max_value = column.max()+1
max_value

In [ ]:
#append sequential number to main file
xx=xx.sort_values(by=['ID'], ascending=[True])
pd_id=pd_id.sort_values(by=['ID'], ascending=[True])
xx =xx.merge(pd_id, on=['ID'], how='inner')
xx.head()

In [ ]:



#sort data
xx=xx.sort_values(by=['ID','DATE'], ascending=[True,True])

#reset index
xx=xx.reset_index(drop=True)

In [ ]:
#create a null dataframe for the next step
df_fred=xx
df_fred['Y_FAIL_sumxx']=0
df_fred=df_fred[df_fred['SCOOBYDOO'] == max_value+1]
df_fred.shape

This step assigns a new failure indicator that incorporates the forecast window.  Note, this calulation occurs at a machine level.  This keeps a signal from one machine effecting another machine.

In [ ]:
for x in range(max_value):
        dffx=xx[xx['SCOOBYDOO'] ==x]
        dff=dffx.copy()
        dff['Y_FAIL_sumxx'] =(dff['Y_FAIL'].rolling(min_periods=1, window=(forecast_window)).sum())
        df_fred= pd.concat([df_fred,dff])

In [ ]:
dff=df_fred
dff.shape

In [ ]:
xx=dff

In [ ]:
# if a signal has occured in the last Z days, the signal is 0.
xx['Y_FAILZ']=np.where((xx.Y_FAIL_sumxx>1), 0, xx.Y_FAIL)

Now, let's examine a confusion matrix with our new failure indicator.

In [ ]:
print(pd.crosstab(xx.Y_FAILZ, xx.EQUIPMENT_FAILURE, dropna=False))

This confusion matrix is more realistic, but we still have more work to do.

A true positive occurs if a failure signal and an actual failure occur on the same day. For example, if a signal appears on Friday and the failure occurs on Saturday, this should be considered a true positive. The confusion matrix above currently counts this as a false positive.

Now, we continue to apply business rules to make the results more realistic. First, we will create a unique id for each signal. This unique id will help us classify them as a true or false prediction of failure.

In [ ]:
#sort the data by id and date.

xx=xx.sort_values(by=['ID','DATE'], ascending=[True, True])

#xx['bootie']=1

In [ ]:
#create signal id with the cumsum function.
xx['SIGNAL_ID'] = xx['Y_FAILZ'].cumsum()




Now we will pull the records with a signal into a different data frame. 

Here we will create a new field that identifies the date of each signal (SIGNAL_DATE). 

Also, we will identify the ID Associated with each signal (ID_OF_SIGNAL)


In [ ]:


df_signals=xx[xx['Y_FAILZ'] == 1]
df_signal_date=df_signals[['SIGNAL_ID','DATE','ID']]
df_signal_date=df_signal_date.rename(index=str, columns={"DATE": "SIGNAL_DATE"})
df_signal_date=df_signal_date.rename(index=str, columns={"ID": "ID_OF_SIGNAL"})




We have a total of 302 signals.  Now each has a unique id.  Note that is still too many.  Below we will apply more heuristics to decrease the number of signals.

In [ ]:
df_signal_date.shape

Append the new fields to the working data frame.

In [ ]:
xx =xx.merge(df_signal_date, on=['SIGNAL_ID'], how='outer')


Keep the fields we need to use going forward.

In [ ]:
xx=xx[['DATE', 'ID', 'EQUIPMENT_FAILURE', 'FAILURE_TARGET','FAILURE_DATE',
       'P_FAIL', 'Y_FAILZ','SIGNAL_ID',
       'SIGNAL_DATE','ID_OF_SIGNAL']]




 Create a field called "Warning" that indicates the time from signal to failure.

In [ ]:

xx['C'] = xx['FAILURE_DATE'] - xx['SIGNAL_DATE']
xx['WARNING'] = xx['C'] / np.timedelta64(1, 'D')


In [ ]:
#Replace nan with 9999.  

xx['WARNING'].fillna(9999, inplace=True)

##  9.0 Define a True Positive, True Negative, False Positive and False Negative <a id="tp"></a>

Defining a false positive, false negative, true positive, and true negative is one of the more essential steps in predictive maintenance models.  The problem's context drives these definitions, and there is no one size fits all approach. 

My definition makes sense here but is unique to this specific business problem.

A true positive occurs if and only if the machine fails, and there was a signal within the last 90 days. Also, we have to ensure that the signal id belongs to the Well ID. Note that this prohibits a signal from another machine from being applied to the machine in question.

A false negative occurs if and only if the machine fails, and it is not a true positive.

A False Positive occurs if there is a failure signal, and a failure does not happen in the next 90 days. Also, if a signal occurs after the failure, this is a false positive. We also have to ensure that the signal ID belongs to the machine ID. Note that this prohibits a signal from another machine from being applied to the machine in question.

If an observation is not a False Positive, a False Negative, or a True Positive, it is a True Negative.


In [ ]:
# define a true positive
xx['TRUE_POSITIVE'] = np.where(((xx.EQUIPMENT_FAILURE == 1) & (xx.WARNING<=forecast_window) &(xx.WARNING>=0) & (xx.ID_OF_SIGNAL==xx.ID)), 1, 0)

In [ ]:
# define a false negative
xx['FALSE_NEGATIVE'] = np.where((xx.TRUE_POSITIVE==0) & (xx.EQUIPMENT_FAILURE==1), 1, 0)

In [ ]:
# define a false positive
xx['BAD_S']=np.where((xx.WARNING<0) | (xx.WARNING>=forecast_window), 1, 0)

xx['FALSE_POSITIVE'] = np.where(((xx.Y_FAILZ == 1) & (xx.BAD_S==1) & (xx.ID_OF_SIGNAL==xx.ID)), 1, 0)

In [ ]:
xx['bootie']=1

In [ ]:
xx['MODELING_GROUP']='TRAINING'

Create the final Cross-Tab

In [ ]:
xx['CATEGORY']=np.where((xx.FALSE_POSITIVE==1),'FALSE_POSITIVE',
                                      (np.where((xx.FALSE_NEGATIVE==1),'FALSE_NEGATIVE',
                                                (np.where((xx.TRUE_POSITIVE==1),'TRUE_POSITIVE','TRUE_NEGATIVE')))))

In [ ]:
table = pd.pivot_table(xx, values=['bootie'], index=['MODELING_GROUP'],columns=['CATEGORY'], aggfunc=np.sum)
table

Using the previously described definitions of a false positive, false negative, true positive and true negative in the training data set there are:
    
    41 False Negatives
    107 False Positives
    106361 True Negatives and
    105 True Postives.
    


## 10.0 Apply Model and Heuristics the Testing and Validation Data Sets. <a id="apply"></a>

Predict the probability of failure for all records.

In [ ]:
df_total['P_FAIL']= xgb0.predict_proba(df_total[features])[:,1];

Create a predicted failure indicator based on a cut-off of .67.

In [ ]:
df_total['Y_FAIL'] = np.where(((df_total.P_FAIL <= .67)), 0, 1)

In [ ]:
yy=df_total




Ensure that failure indicator occurs only once every 120 days.

In [ ]:
aa=yy

pd_id=aa.drop_duplicates(subset='ID')
pd_id=pd_id[['ID']]
pd_id.shape


In [ ]:
pd_id=pd_id.reset_index(drop=True)
pd_id=pd_id.reset_index(drop=False)
pd_id=pd_id.rename(columns={"index": "SCOOBYDOO"})
pd_id['SCOOBYDOO']=pd_id['SCOOBYDOO']+1
pd_id.head()

In [ ]:

column = pd_id["SCOOBYDOO"]
max_value = column.max()+1
max_value

In [ ]:
yy=yy.sort_values(by=['ID'], ascending=[True])
pd_id=pd_id.sort_values(by=['ID'], ascending=[True])
yy =yy.merge(pd_id, on=['ID'], how='inner')
yy.head()

In [ ]:
yy=yy.sort_values(by=['ID','DATE'], ascending=[True,True])

In [ ]:
yy=yy.reset_index(drop=True)

In [ ]:
df_fred=yy
df_fred['Y_FAIL_sumxx']=0
df_fred=df_fred[df_fred['SCOOBYDOO'] == max_value+1]
df_fred.shape

In [ ]:
for x in range(max_value):
        dffx=yy[yy['SCOOBYDOO'] ==x]
        dff=dffx.copy()
        dff['Y_FAIL_sumxx'] =(dff['Y_FAIL'].rolling(min_periods=1, window=(forecast_window)).sum())
        df_fred= pd.concat([df_fred,dff])
        

In [ ]:
dff=df_fred
dff.shape

In [ ]:
yy=dff

In [ ]:
yy['Y_FAILZ']=np.where((yy.Y_FAIL_sumxx>1), 0, yy.Y_FAIL)

Create the WARNING Field

In [ ]:
yy=yy.sort_values(by=['ID','DATE'], ascending=[True, True])

In [ ]:
#create a signal id
yy['SIGNAL_ID'] = yy['Y_FAILZ'].cumsum()



In [ ]:
#create the signal date and ID_OF_SIGNAL

yy_signals=yy[yy['Y_FAILZ'] == 1]
yy_signal_date=yy_signals[['SIGNAL_ID','DATE','ID']]
yy_signal_date=yy_signal_date.rename(index=str, columns={"DATE": "SIGNAL_DATE"})
yy_signal_date=yy_signal_date.rename(index=str, columns={"ID": "ID_OF_SIGNAL"})

In [ ]:
#merge the two data frames back into one.

yy =yy.merge(yy_signal_date, on=['SIGNAL_ID'], how='outer')


In [ ]:
#Keep on the fields we need
yy=yy[['DATE', 'ID', 'EQUIPMENT_FAILURE', 'FAILURE_TARGET','FAILURE_DATE','MODELING_GROUP',
       'P_FAIL', 'Y_FAILZ','SIGNAL_ID',
       'SIGNAL_DATE','ID_OF_SIGNAL']]

In [ ]:
# Calculate the warning time between each failure date and signal date.
yy['C'] = yy['FAILURE_DATE'] - yy['SIGNAL_DATE']
yy['WARNING'] = yy['C'] / np.timedelta64(1, 'D')

Define True Positives, True Negatives, False Positives and False Negatives.

In [ ]:
yy['WARNING'].fillna(9999, inplace=True)


In [ ]:
# define a true positive
yy['TRUE_POSITIVE'] = np.where(((yy.EQUIPMENT_FAILURE == 1) & (yy.WARNING<=forecast_window) &(yy.WARNING>=0) & (yy.ID_OF_SIGNAL==yy.ID)), 1, 0)

In [ ]:
# define a false negative
yy['FALSE_NEGATIVE'] = np.where((yy.TRUE_POSITIVE==0) & (yy.EQUIPMENT_FAILURE==1), 1, 0)

In [ ]:
# define a false positive
yy['BAD_S']=np.where((yy.WARNING<0) | (yy.WARNING>=forecast_window), 1, 0)

yy['FALSE_POSITIVE'] = np.where(((yy.Y_FAILZ == 1) & (yy.BAD_S==1) & (yy.ID_OF_SIGNAL==yy.ID)), 1, 0)

In [ ]:
yy['bootie']=1

In [ ]:
yy['CATEGORY']=np.where((yy.FALSE_POSITIVE==1),'FALSE_POSITIVE',
                                      (np.where((yy.FALSE_NEGATIVE==1),'FALSE_NEGATIVE',
                                                (np.where((yy.TRUE_POSITIVE==1),'TRUE_POSITIVE','TRUE_NEGATIVE')))))

Define metrics for the Testing, Training and Validation Data sets.

In [ ]:
table = pd.pivot_table(yy, values=['bootie'], index=['MODELING_GROUP'],columns=['CATEGORY'], aggfunc=np.sum)
table

Remember when I introduced the use case I presented this chart.


In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/4800/1*5Ar2n3ZHMwVeOhgXI_dm0w.png")

A false positive is “Unnecessary Maintenance.” 
A true positive is a “Timely and Appropriate Maintenance.” 
A false negative is “Machine Runs to Failure.” 

This means that a false positive costs $1,500.   

A false negative costs $30,000.   

A true positive costs $7,500. 

A true negative has no cost because no action is taken.

Now we can calculate the total cost.

In [ ]:
yy['TOTAL_COST']=yy.FALSE_NEGATIVE*30000+yy.FALSE_POSITIVE*1500+yy.TRUE_POSITIVE*7500

Aggregate the costs by modeling group

In [ ]:

table = pd.pivot_table(yy, values=['TOTAL_COST'],index=['MODELING_GROUP'], aggfunc=np.sum)
table

Calculate the number of machines per modelling group

In [ ]:
wells=yy[['ID','MODELING_GROUP']]

In [ ]:
wells=wells.drop_duplicates(subset='ID')

wells.shape

In [ ]:
wells = wells.groupby(['MODELING_GROUP'])['ID'].count()
wells=pd.DataFrame(wells)
wells=wells.rename(columns={"ID": "WELLS"})

In [ ]:
wells

Merge the total costs and total machines into one dataframe

In [ ]:
tc = yy.groupby(['MODELING_GROUP'])['TOTAL_COST'].sum()
tc=pd.DataFrame(tc)


Calculate the average cost per machine

In [ ]:
ac =tc.merge(wells, on=['MODELING_GROUP'], how='inner')


In [ ]:
ac['AVERAGE_COST']=ac.TOTAL_COST/ac.WELLS
ac['LIFT']=28000-ac.AVERAGE_COST

In [ ]:
ac

##  11.0 Conclusions <a id="conc"></a>

It currently costs the firm about 28,000 dollars per machine in the current data set. By deploying a PM solution, we can lower those costs to between 22,000 and 23,000 dollars. This equates to a savings of about 4,500 per machine. For all 419 machines, this is a total savings of approximately 1,885,000 dollars.

Not too shabby.

One final note. There are many judgments I made that work for this example but may not work for you. Unfortunately, there is no “one size fits all” solution for any data science problem.

Nonetheless, this exercise should give you a useful reference as you approach these types of problems in the future.

As far as the next steps, I would encourage you to see if you can improve the solution by optimizing the model. Maybe incorporate some hyper-parameter optimization or even try a different model. Let me know how it turns out!



### Author





**Shad Griffin**, is a Data Scientist at the IBM Global Solution Center in Dallas, Texas

<hr>
Copyright &copy; IBM Corp. 2020. This notebook and its source code are released under the terms of the MIT License.

